In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/datasets/BankChurn/bank_data_without_jobs.csv',
                 index_col='ID'); df

,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,TURNOVER_DYNAMIC_IL_1M,AMOUNT_RUB_SUP_PRC,REST_DYNAMIC_FDEP_1M,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,AMOUNT_RUB_NAS_PRC,TRANS_COUNT_SUP_PRC,...,PACK_103,PACK_104,PACK_105,PACK_107,PACK_108,PACK_109,PACK_301,PACK_k01,PACK_m01,PACK_o01
ID,,,,,,,,,,,,,,,,,,,,,
146841,-0.243932,-0.380232,-0.044824,-0.582347,-0.05133,2.356500,-0.169589,0.477531,-0.248485,-1.004598,...,0,0,0,0,0,0,0,1,0,0
146842,-0.243932,0.016835,-0.044824,1.221828,-0.05133,-0.312371,-0.169589,-0.330458,-0.248485,1.446347,...,0,0,0,0,0,0,0,0,0,0
146843,-0.243932,-0.313313,-0.044824,-0.582347,-0.05133,-0.312371,-0.169589,0.247411,-0.248485,-1.004598,...,0,0,0,0,0,0,0,0,0,0
146844,-0.243932,-0.380232,-0.044824,-0.582347,-0.05133,-0.283428,-0.169589,-0.124857,-0.248485,-1.004598,...,0,0,0,0,0,0,0,1,0,0
146845,-0.243932,-0.380232,-0.044824,-0.582347,-0.05133,-0.312371,-0.169589,0.029532,-0.248485,-1.004598,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590822,-0.243932,-0.380232,-0.044824,-0.582347,-0.05133,-0.312371,-0.169589,-0.301613,-0.248485,-1.004598,...,0,1,0,0,0,0,0,0,0,0
590823,-0.243932,-0.380232,-0.044824,-0.582347,-0.05133,-0.312371,-0.169589,1.927086,2.804227,-1.004598,...,0,1,0,0,0,0,0,0,0,0
590825,-0.243932,0.019401,-0.044824,0.119528,-0.05133,-0.312371,-0.169589,0.843853,0.113815,0.135764,...,0,0,0,0,0,0,0,1,0,0


In [4]:
X = df.drop(columns = 'TARGET')

In [5]:
y = df.TARGET

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33,
                                                    random_state=8,
                                                   shuffle=True,
                                                   stratify=y)

Class weight calculation

In [7]:
class_weights = (1 - y.value_counts(normalize=True)).to_dict()
class_weights

{0: 0.08143528815563505, 1: 0.918564711844365}

Initial bias

In [15]:
neg = y.value_counts()[0]
pos = y.value_counts()[1]

In [16]:
initial_bias = np.log([pos/neg])
initial_bias

array([-2.42300366])

In [17]:
output_bias = tf.keras.initializers.Constant(initial_bias)

In [19]:
metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid', 
                          bias_initializer=output_bias),

]) 


In [21]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=metrics)

In [22]:
history = model.fit(
   x=X_train,
   y=y_train,
   epochs=30,
   validation_split=0.33,
   class_weight=class_weights
)

Epoch 1/30
4983/4983 [==============================] - 34s 7ms/step - loss: 0.0962 - accuracy: 0.6947 - precision: 0.1508 - recall: 0.5882 - auc: 0.7066 - val_loss: 0.5659 - val_accuracy: 0.6961 - val_precision: 0.1705 - val_recall: 0.7203 - val_auc: 0.7763
Epoch 2/30
4983/4983 [==============================] - 32s 6ms/step - loss: 0.0880 - accuracy: 0.6914 - precision: 0.1645 - recall: 0.6776 - auc: 0.7554 - val_loss: 0.5505 - val_accuracy: 0.7075 - val_precision: 0.1769 - val_recall: 0.7235 - val_auc: 0.7888
Epoch 3/30
4983/4983 [==============================] - 31s 6ms/step - loss: 0.0851 - accuracy: 0.6908 - precision: 0.1697 - recall: 0.7118 - auc: 0.7725 - val_loss: 0.5866 - val_accuracy: 0.6495 - val_precision: 0.1608 - val_recall: 0.7975 - val_auc: 0.7912
Epoch 4/30
4983/4983 [==============================] - 31s 6ms/step - loss: 0.0833 - accuracy: 0.6829 - precision: 0.1708 - recall: 0.7436 - auc: 0.7827 - val_loss: 0.5562 - val_accuracy: 0.6658 - val_precision: 0.1661 - v

In [27]:
model.evaluate(X_test, y_test)

3663/3663 [==============================] - 14s 4ms/step - loss: 0.5336 - accuracy: 0.6438 - precision: 0.1619 - recall: 0.8081 - auc: 0.7916


[0.533601701259613,
 0.6438364386558533,
 0.16194255650043488,
 0.8080670237541199,
 0.7915567755699158]